In [191]:
import torch
from fontTools import ttLib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import time
import os
from tqdm import tqdm
import shutil
import time
import numpy as np
import seaborn
from shape import from_svg_path
from xml.dom import minidom
import bezier
import pydiffvg
import pickle as pkl

ModuleNotFoundError: No module named 'pydiffvg'

In [13]:
FONTDIR = "fonts/"
PATHS = [path[len(FONTDIR):] for path in glob.glob(f'{FONTDIR}/*')] # list of font names
SVG_OUTPUT_PATH = "./SVGs"      # svg output directory
BEZIER_OUTPUT_PATH = "./Bezier" # Bezier output dir
PATHS

['NotoSansCJKtc-Regular.otf',
 'fontA.ttf',
 'AdobeSongStd-Light.otf',
 'AdobeFangsongStd-Regular.otf',
 'AdobeHeitiStd-Regular.otf',
 'AdobeKaitiStd-Regular.otf']

In [14]:
shutil.rmtree(f'{SVG_OUTPUT_PATH}') #cleans OUTPUT_PATH and also removes it; will be added back in on the next for loop
print(f'Finished Cleaning {SVG_OUTPUT_PATH}')
for path in PATHS:  
    #ensure every font has a corresponding folder, name of folder is just font file name
    newpath = f'{SVG_OUTPUT_PATH}/{path}'
    try:
        os.makedirs(newpath)
    except:
        continue
print("Finished Reconstructing Folders")


Finished Cleaning ./SVGs
Finished Reconstructing Folders


In [15]:
fonts = [ttLib.TTFont(FONTDIR + path) for path in PATHS]
chars = pd.read_csv("./edu_standard.csv")

In [18]:
supports = np.zeros((len(PATHS), len(chars["Character"])))                # HasFound[i][j]: If the ith font supports character j
# indexes mentioned in the code are all the index that they appear in the edu_standard.csv file.
for i, font in enumerate(fonts): 
    for cmap in font['cmap'].tables:
        if cmap.isUnicode():
            st = time.time()
            toGenerate = []  # List of characters supported by the font
            indexes    = []  # The indexes of the characters in the above
            for idx, char_ in enumerate(chars["Character"]):
                char = ord(char_)
                if char in cmap.cmap:
                    toGenerate.append(cmap.cmap[char]) # cmap.cmap[char] stores the CID for the given character
                    indexes.append(idx)
                supports[i, idx] = (char in cmap.cmap) 

            # Generate desired svg files
            os.system(f'fonts2svg -c 000000 {FONTDIR}{PATHS[i]} -o {SVG_OUTPUT_PATH}/{PATHS[i]} -g {",".join(toGenerate)} > /dev/null') 

            for idx, svgName in zip(indexes, toGenerate):
                os.rename(f'{SVG_OUTPUT_PATH}/{PATHS[i]}/{svgName}.svg', f'{SVG_OUTPUT_PATH}/{PATHS[i]}/{idx}.svg') # rename according to index
            ed = time.time()
            print(f'Finished Generation for font {PATHS[i]}: ΔT = {ed - st}s')
            break
            
with open("./font_support.npy", "wb") as f:
    np.save(f, supports)
    print("Saved Font Support Matrix to disk")

    

11151it [00:00, 685455.48it/s]
11151it [00:01, 9554.77it/s] 


Finished Generation for font NotoSansCJKtc-Regular.otf: ΔT = 14.379251956939697s


11151it [00:00, 1653726.18it/s]
6322it [00:00, 10231.10it/s]


Finished Generation for font fontA.ttf: ΔT = 10.93431305885315s


11151it [00:00, 1607737.24it/s]
11151it [00:00, 18879.18it/s]


...

Finished Generation for font AdobeFangsongStd-Regular.otf: ΔT = 10.832360029220581s


11151it [00:00, 1629527.00it/s]
11151it [00:00, 16962.35it/s]


Finished Generation for font AdobeHeitiStd-Regular.otf: ΔT = 10.811652660369873s


11151it [00:00, 1651273.97it/s]
11151it [00:00, 17546.76it/s]

Finished Generation for font AdobeKaitiStd-Regular.otf: ΔT = 11.516471147537231s
Saved Font Support Matrix to disk


AttributeError: module 'pickle' has no attribute 'save'

In [188]:

def getBeziers(svg_path, cut_parts = 8):
    """
    Takes an SVG file, and adds 8* anchor points to it, while elevating all curves into cubic ones. 
    output: a list of diffvg Paths. 
    
    @param svg_path:  the path to the svg file to be cut
    @param cut_parts: the factor of anchor points to add
    
    @return res:      a list of diffvg paths to be used in ML models
    """
    mydoc = minidom.parse(svg_path)
    path_tag = mydoc.getElementsByTagName("path")
    d_string = path_tag[0].attributes['d'].value
    x = from_svg_path(d_string) 
    #y = pydiffvg.svg_to_scene(path_tag)
    #print(y)
    for curve in x:
        paths = []
        points = curve.points.clone()
        num_control_points = curve.num_control_points.clone()
        points = torch.cat([points, points[0].unsqueeze(0)], dim = 0) # push element 0 to last point
        anchors = np.cumsum(num_control_points.clone() + 1) 
        anchors = torch.cat([torch.tensor([0]), anchors], dim = 0)
        # anchors[i] and anchors[i + 1] are indices of anchors, anything in between are control points
        for idx in range(len(anchors) - 1):
            bezierCurve = bezier.Curve(points[anchors[idx] : anchors[idx + 1] + 1, :].T, degree = anchors[idx + 1] - anchors[idx])
            while bezierCurve.degree < 3: # Make sure everything is cubic
                bezierCurve = bezierCurve.elevate()
            # Split curve into cut_parts parts 
            parts = []
            for i in range(cut_parts):
                parts.append(bezierCurve.specialize(i / 8, (i + 1) / 8))
            paths = np.concatenate([paths, parts], axis = 0)
        
        newpoints = np.empty((0, 2))
        new_controls = np.empty(0)
        
        for path in paths: # get paths back into diffsvg format
            newpoints = np.concatenate([newpoints, path.nodes.T[ : -1, :]], axis = 0)
            new_controls = np.concatenate([new_controls, [path.degree - 1]])
        
        curve.points = torch.tensor(newpoints)
        curve.num_control_points = torch.tensor(new_controls)
    
    return x
 
    



In [189]:
seaborn.set()
x = getBeziers("./SVGs/AdobeFangsongStd-Regular.otf/47.svg")

for curve in x:
    print(len(curve.points))
    print(curve.points[0])
    print(curve.num_control_points.shape)

1584
tensor([ 922., -301.], dtype=torch.float64)
torch.Size([528])
96
tensor([ 707., -575.], dtype=torch.float64)
torch.Size([32])
96
tensor([ 484., -480.], dtype=torch.float64)
torch.Size([32])
96
tensor([ 714., -370.], dtype=torch.float64)
torch.Size([32])
96
tensor([ 703., -268.], dtype=torch.float64)
torch.Size([32])


In [180]:
def graphFromBezier(svg_path):
    """
    @param svg_path      : the path of the svg file
    
    @return node_feature : Node feature matrix of shape [num_nodes, in_channels]
    @return edge_index   : Graph connectivity matrix of shape [2, num_edges]
    
    return values are all numpy arrays and should be turned into Tensors later
    """
    mydoc = minidom.parse(svg_path)
    path_tag = mydoc.getElementsByTagName("path")
    d_string = path_tag[0].attributes['d'].value
    x = from_svg_path(d_string) 
    """ debug mode: comment out
    for curve in x:
        for dim in curve.num_control_points:
            assert(dim == 2)
    """
    num_nodes = 0
    indices = []
    for curve in x:
        indices.append(np.arange(num_nodes, num_nodes + len(curve.points)))
        num_nodes += len(curve.points)
        
    node_feature = np.zeros((num_nodes, 3)) # position, is anchor
    edge_index = [] #will have to transpose 
    
    for i, curve in enumerate(x):
        for j, point in enumerate(curve.points):
            feat = np.zeros(3) # list of length 3
            feat[ 0 : 2 ] = point
            feat[2] = 1 if j % 3 == 0 else 0 #assumes that all paths are closed cubic paths!!
            node_feature[indices[i][j]] = feat
            
            if(j % 3 == 0):
                edge_index.append(np.array((indices[i][j], indices[i][(j + 3) % len(curve.points)])))
            edge_index.append(np.array((indices[i][j], indices[i][(j + 1) % len(curve.points)])))
            
    return node_feature, np.array(edge_index).T

In [181]:
graphFromBezier("./SVGs/AdobeFangsongStd-Regular.otf/47.svg")

(array([[ 922., -301.,    1.],
        [ 881., -334.,    0.],
        [ 864., -326.,    0.],
        [ 821., -315.,    1.],
        [ 749., -306.,    0.],
        [ 754., -345.,    0.],
        [ 757., -371.,    1.],
        [ 767., -385.,    0.],
        [ 780., -391.,    0.],
        [ 785., -393.,    1.],
        [ 785., -397.,    0.],
        [ 780., -403.,    0.],
        [ 742., -445.,    1.],
        [ 713., -405.,    0.],
        [ 524., -384.,    0.],
        [ 524., -451.,    1.],
        [ 752., -476.,    0.],
        [ 763., -477.,    0.],
        [ 764., -479.,    1.],
        [ 757., -488.,    0.],
        [ 736., -513.,    0.],
        [ 744., -557.,    1.],
        [ 746., -569.,    0.],
        [ 751., -576.,    0.],
        [ 767., -585.,    1.],
        [ 774., -589.,    0.],
        [ 773., -593.,    0.],
        [ 768., -599.,    1.],
        [ 732., -639.,    0.],
        [ 702., -606.,    0.],
        [ 524., -589.,    1.],
        [ 524., -659.,    0.],
        